## DIA 119: Permitir Descargar Todo el Historial de Reportes como un Archivo ZIP

Hoy agregarás una funcionalidad muy útil: el usuario podrá descargar todos sus reportes (PDF y CSV) en un solo archivo comprimido .zip.

✅ ¿Qué aprenderás hoy?

📂 Buscar todos los archivos de reportes del usuario

📦 Crear un archivo ZIP dinámicamente en memoria

📥 Enviar el ZIP como descarga directa

🧩 1. Ruta Flask para generar y enviar el ZIP
python
Copiar
Editar
from flask import send_file
from zipfile import ZipFile
import tempfile
import os

@app.route('/descargar-historial-zip')
@login_required
def descargar_historial_zip():
    reportes = Reporte.query.filter_by(user_id=current_user.id).all()
    if not reportes:
        return "No hay reportes disponibles para descargar."

    # Crear archivo ZIP en memoria
    tmp_zip = tempfile.NamedTemporaryFile(delete=False, suffix='.zip')
    with ZipFile(tmp_zip.name, 'w') as zipf:
        for reporte in reportes:
            try:
                # Descargar el archivo desde la URL
                response = requests.get(reporte.url)
                extension = reporte.tipo
                nombre = f"{reporte.fecha_generado.strftime('%Y%m%d_%H%M')}.{extension}"
                zipf.writestr(nombre, response.content)
            except Exception as e:
                print(f"❌ Error al agregar {reporte.url} al ZIP: {e}")

    # Volver a abrir el archivo ZIP para enviar
    return send_file(tmp_zip.name, as_attachment=True, download_name='historial_reportes.zip')
🔐 Este método descarga desde las URLs públicas (como Firebase). Si usas rutas locales, simplemente apunta a open(ruta_local).

🧩 2. Añadir botón de descarga en la interfaz
Agrega este botón en tu historial (historial_reportes.html):

html
Copiar
Editar
<a href="/descargar-historial-zip">
  <button>📦 Descargar Todos los Reportes (.zip)</button>
</a>